In [1]:
import yaml
import os
from kubernetes import client, config
from typing import List, Optional


def path_to_config() -> str:
    return os.getenv("HOME") if os.getenv("HOME") else os.getenv("HOMEPATH")


def load_ctx_and_ns() -> (str, str):
    """Loads current context from .kube/config.yaml"""
    config_path = os.path.join(path_to_config(), ".kube", "config")
    with open(config_path, "r") as fin:
        kube_config = yaml.safe_load(fin)
        context: str = kube_config.get("current-context")       
        contexts = kube_config.get("contexts")
        namespace: List[str] = [c.get("context").get("namespace")
                         for c in contexts 
                         if c.get("name") == context and c.get("context")]
        return context, namespace[0]


In [2]:
# Init
ctx, ns = load_ctx_and_ns()
ac = config.new_client_from_config()

In [3]:
def list_pods(namespace: str, api_client: client.api_client.ApiClient) -> client.V1PodList:
    api_instance = client.CoreV1Api(api_client)
    return api_instance.list_namespaced_pod(namespace)


p = list_pods(ns, ac)

In [4]:
def list_replicasets(namespace: str, api_client: client.api_client.ApiClient) -> client.V1ReplicaSetList:
    api_instance = client.AppsV1Api(api_client)
    return api_instance.list_namespaced_replica_set(namespace)


rs = list_replicasets(ns, ac)

In [5]:
def list_deployments(namespace: str, api_client: client.api_client.ApiClient) -> client.V1DeploymentList:
    api_instance = client.AppsV1Api(api_client)
    return api_instance.list_namespaced_deployment(namespace)


d = list_deployments(ns, ac)

In [6]:
def get_replicaset_name(pod: client.V1Pod) -> Optional[str]:
    if pod and pod.metadata and len(pod.metadata.owner_references) > 0:
        return pod.metadata.owner_references[0].name
    else:
        return None


def get_deployment_name(replicaset: client.V1ReplicaSet) -> Optional[str]:
    rs = replicaset # Shorter name
    if rs and rs.metadata and len(rs.metadata.owner_references) > 0:
        return rs.metadata.owner_references[0].name
    else:
        return None  


In [7]:
def find_pods_for_deployment(deployment_name, namespace: str, api_client: client.api_client.ApiClient) -> List[client.V1Pod]:
    rs_list = list_replicasets(namespace, api_client)
    rs_names = [rs.metadata.name 
               for rs in rs_list.items 
               if get_deployment_name(rs) == deployment_name]
    
    pods = list_pods(namespace, api_client)
    return [p 
            for p in pods.items
            if get_replicaset_name(p) in rs_names]


fp = find_pods_for_deployment("jarvis-deployment", ns, ac)